In [1]:
# %pip install numpy
# %pip install pandas
# %pip install torch

In [2]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub


In [3]:
# %pip install llama-cpp-python

In [2]:
from huggingface_hub import hf_hub_download

In [5]:
from llama_cpp import Llama

In [6]:
#download the model llama 2 13b chat gguf
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is gguf format

In [7]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [8]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rop

llama_model_loader: - kv  12:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   81 tensors
llama_model_loader: - type q5_K:  281 tensors
llama_model_loader: - type q6_K:    1 tensors
llm_load_vocab: special tokens definition check successful ( 259/32000 ).
llm_load_prin

In [9]:

prompt= f'''2 / 3 = ?'''

prompt_template=f'''SYSTEM: You are a math assistant.I will ask you some division questions. Please answer upto 12 digits after decimal point.For example, if I ask you '0.9 / 0.1 = ?', you should answer 'answer=9.000000000000'. Do not include any other information in your answer.

USER: {prompt}
        
ASSISTANT:'''
        

In [10]:
response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)


llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =      13.65 ms /    34 runs   (    0.40 ms per token,  2490.30 tokens per second)
llama_print_timings: prompt eval time =    4800.76 ms /    99 tokens (   48.49 ms per token,    20.62 tokens per second)
llama_print_timings:        eval time =   22777.71 ms /    33 runs   (  690.23 ms per token,     1.45 tokens per second)
llama_print_timings:       total time =   27671.08 ms /   132 tokens


In [11]:
print(response["choices"][0]["text"])

 2/3 = 0.6666666666667 (answer upto 12 digits after decimal point)


In [12]:
import json

In [13]:
#load file in data folder
with open('division_dataset.json') as f:
    data = json.load(f)

In [14]:
#data as pandas dataframe
import pandas as pd

df = pd.DataFrame(data)
#last 5 rows
df.tail()

,0,1,2,3,4
3427,-665059.0,-3132.0,212.3432,212.343231,212.343231
3428,-96240.0,409482.0,-0.2350,-0.235029,-0.235029
3429,409482.0,-96240.0,-4.2548,-4.254800,-4.254800
3430,-96240.0,-409482.0,0.2350,0.235029,0.235029
3431,-409482.0,-96240.0,4.2548,4.254800,4.254800


In [15]:
print(df.head())

     0    1       2         3         4
0  1.0  2.0  0.5000  0.500000  0.500000
1  2.0  1.0  2.0000  2.000000  2.000000
2  1.0  3.0  0.3333  0.333333  0.333333
3  3.0  1.0  3.0000  3.000000  3.000000
4  1.0  4.0  0.2500  0.250000  0.250000


In [16]:
print(len(df))

3432


In [17]:
prompt= f"{df.iloc[1, 0]} / {df.iloc[1, 1]} = ?"
print(prompt)

2.0 / 1.0 = ?


In [18]:
import time



for i in range(0, len(df)):
    
    if i%100==0:
        print(f"Completed {i} rows")
        
    try:
        
        prompt= f"{df.iloc[i, 0]} / {df.iloc[i, 1]} = ?"


        prompt_template=f'''SYSTEM: You are a math assistant.I will ask you some division questions. Please answer upto 12 digits after decimal point.For example, if I ask you '0.9 / 0.1 = ?', you should answer 'answer=9.000000000000'. Do not include any other information in your answer.
        
        USER: {prompt}
        
        ASSISTANT:'''
        
        response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=False)
        
        with open('division_12_digit.txt', 'a') as f:
            answer=response["choices"][0]["text"]
            f.write(f"{df.iloc[i, 0]} / {df.iloc[i, 1]} = {answer} \n")
            
            
    except:
        
        with open('division_12_digit.txt', 'a') as f:
            f.write(f"{df.iloc[i, 0]} / {df.iloc[i, 1]} = Error \n")
            
        time.sleep(2)

Completed 0 rows


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.84 ms /    18 runs   (    0.38 ms per token,  2632.35 tokens per second)
llama_print_timings: prompt eval time =    1315.91 ms /    26 tokens (   50.61 ms per token,    19.76 tokens per second)
llama_print_timings:        eval time =   11411.22 ms /    17 runs   (  671.25 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   12775.06 ms /    43 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       7.23 ms /    17 runs   (    0.43 ms per token,  2350.99 tokens per second)
llama_print_timings: prompt eval time =    1025.22 ms /    19 tokens (   53.96 ms per token,    18.53 tokens per second)
llama_print_timings:        eval time =   10956.50 ms /    16 runs   (  684.78 ms per token,     1.46 tokens per second)
llama_print_timings:       to

Completed 100 rows



llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.34 ms /    17 runs   (    0.37 ms per token,  2683.08 tokens per second)
llama_print_timings: prompt eval time =    1002.16 ms /    19 tokens (   52.75 ms per token,    18.96 tokens per second)
llama_print_timings:        eval time =   10674.81 ms /    16 runs   (  667.18 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =   11723.23 ms /    35 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.99 ms /    18 runs   (    0.39 ms per token,  2576.58 tokens per second)
llama_print_timings: prompt eval time =     984.21 ms /    19 tokens (   51.80 ms per token,    19.30 tokens per second)
llama_print_timings:        eval time =   11320.87 ms /    17 runs   (  665.93 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =   12353.83 ms /    36 

Completed 200 rows


...


llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.49 ms /    17 runs   (    0.38 ms per token,  2617.80 tokens per second)
llama_print_timings: prompt eval time =    1479.32 ms /    30 tokens (   49.31 ms per token,    20.28 tokens per second)
llama_print_timings:        eval time =   10729.31 ms /    16 runs   (  670.58 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   12257.71 ms /    46 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.98 ms /    18 runs   (    0.39 ms per token,  2578.43 tokens per second)
llama_print_timings: prompt eval time =    1465.22 ms /    30 tokens (   48.84 ms per token,    20.47 tokens per second)
llama_print_timings:        eval time =   11196.21 ms /    17 runs   (  658.60 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =   12713.60 ms /    47 

Completed 3300 rows



llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =     113.34 ms /   395 runs   (    0.29 ms per token,  3484.97 tokens per second)
llama_print_timings: prompt eval time =    1370.84 ms /    28 tokens (   48.96 ms per token,    20.43 tokens per second)
llama_print_timings:        eval time =  266312.66 ms /   394 runs   (  675.92 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =  268885.53 ms /   422 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       6.65 ms /    17 runs   (    0.39 ms per token,  2555.62 tokens per second)
llama_print_timings: prompt eval time =    1435.62 ms /    29 tokens (   49.50 ms per token,    20.20 tokens per second)
llama_print_timings:        eval time =   10756.25 ms /    16 runs   (  672.27 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   12241.78 ms /    45 

Completed 3400 rows



llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       7.78 ms /    20 runs   (    0.39 ms per token,  2571.02 tokens per second)
llama_print_timings: prompt eval time =    1423.15 ms /    29 tokens (   49.07 ms per token,    20.38 tokens per second)
llama_print_timings:        eval time =   12571.19 ms /    19 runs   (  661.64 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =   14052.64 ms /    48 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4801.15 ms
llama_print_timings:      sample time =       7.02 ms /    18 runs   (    0.39 ms per token,  2563.74 tokens per second)
llama_print_timings: prompt eval time =    1438.37 ms /    29 tokens (   49.60 ms per token,    20.16 tokens per second)
llama_print_timings:        eval time =   11325.03 ms /    17 runs   (  666.18 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =   12816.27 ms /    46 